### Preprocess CD4 Cropseq Dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scanpy.api as sc
import scipy as sp
import itertools
import numpy as np
import scipy.stats as stats
from scipy.integrate import dblquad
import seaborn as sns
from statsmodels.stats.multitest import fdrcorrection
import imp
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)
import pickle as pkl
import time

In [2]:
data_path = '/wynton/group/ye/mincheol/parameter_estimation/'

In [3]:
with open(data_path + 'interferon_data/immune_genes.pkl', 'rb') as f:
    immune_genes = pkl.load(f)

### Select relevant cells and save the AnnData object

In [4]:
adata = sc.read('/wynton/group/ye/rgate/072319_tfko_140_seq/tfko.sng.guides.h5ad')

Only considering the two last: ['.guides', '.h5ad'].
Only considering the two last: ['.guides', '.h5ad'].


In [5]:
df = pd.read_csv(
    '/wynton/group/ye/rgate/072319_tfko_140_seq/tfko.sng.guides.combat.n50.res1.meta.txt', 
    sep='\t',
    index_col=0)

In [6]:
df_singlet = df.query('guide_multiplet_cov == "1"').copy()

In [7]:
df_singlet['group'] = df_singlet.apply(lambda row: '.'.join(row['guide1_cov'].split('.')[:-1]) if row['WT'] == 'F' else 'WT', axis=1)

In [9]:
adata_singlets = adata[df_singlet.index].copy()
adata_singlets.obs['group'] = df_singlet['group']
adata_singlets.obs['n_counts'] = df_singlet['n_counts']

In [10]:
ko_genes = list(set([x.split('.')[0] for x in adata_singlets.obs.guide1_cov.drop_duplicates().tolist()]))

In [11]:
genes = list(set(ko_genes) | set(immune_genes))

In [12]:
adata_singlets = adata_singlets[:, genes].copy()

In [13]:
adata_singlets.shape

(105203, 562)

In [14]:
sc.write(data_path + 'cd4_cropseq_data/guide_singlets.h5ad', adata_singlets)

... storing 'group' as categorical


### Get a list of genes to test correlations against

In [57]:
prefixes = ['STAT', 'IRF', 'IFI', 'ETS', 'NCOA', 'JUN', 'BATF', 'SATB1', 'FOXP']
genes_to_test = []
for gene in ko_genes:
    for prefix in prefixes:
        if prefix in gene:
            genes_to_test.append(gene)

In [59]:
with open(data_path + 'cd4_cropseq_data/genes_to_test.pkl', 'wb') as f:
    pkl.dump(genes_to_test, f)